In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
#from vso import util, data
import astropy.units as u
from SetupSession import use_dark_theme


In [ ]:
import vso.util
import vso.data

OBJ_NAME='SZ Lyn'
SESSION_TAG='20250126'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vso.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vso.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
IMAGE_DIR = vso.util.ImageLayout(IMAGE_ROOT).get_images(session).lights_dir / 'V'
sd = vso.data.StarData(layout.charts_dir)

In [ ]:
import ccdproc as ccdp
import vso.reduce

ifc = ccdp.ImageFileCollection(IMAGE_DIR)
path = IMAGE_DIR / ifc.summary['file'][0]
image  = vso.reduce.load_and_solve(path, session_layout.solved_dir)
matcher = vso.reduce.CalibrationMatcher(layout.calibr_dir, temp_tolerance=2*u.K)
cal = matcher.match(image.header)
reduced = vso.reduce.calibrate_image(image,
                        dark=cal.dark,
                        flat=cal.flat)


In [ ]:
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from astropy.coordinates import SkyCoord

mean, median, std = sigma_clipped_stats(reduced.data, sigma=3.0)
daofind = DAOStarFinder(fwhm=10.0, threshold=5.*std)

sources = daofind(reduced.data)
sources


In [ ]:
from photutils.aperture import CircularAperture, ApertureStats

ap = CircularAperture(zip(sources['xcentroid'], sources['ycentroid']), r=5)
ph = ApertureStats(reduced, ap)
ph.ellipticity

In [ ]:
import matplotlib.pyplot as plt
ax = plt.subplot()

#c = ax.tricontourf(sources['xcentroid'], sources['ycentroid'], sources['roundness2'])
c = ax.tricontourf(ph.xcentroid, ph.ycentroid, ph.ellipticity.value)
plt.colorbar(c, ax=ax)

plt.show()

In [ ]:
import vso.phot
from astropy.table import QTable

stars = QTable(dict(sky_centroid=reduced.wcs.pixel_to_world(sources['xcentroid'], sources['ycentroid']),
                    auid=sources['id']))

stars = vso.phot.measure_photometry(reduced, stars, 5*u.arcsec, (10*u.arcsec, 15*u.arcsec))
stars.rename_columns(['sky_centroid'], ['radec2000'])
stars

In [ ]:
chart = sd.get_chart(image.header['OBJECT'], fov=60 * u.arcsec)['auid', 'radec2000', image.header['FILTER']]
chart

In [ ]:
from astropy.table import join, join_skycoord
chart_stars = join(stars, chart, keys='radec2000', join_funcs={'radec2000': join_skycoord(2 * u.arcsec)})
chart_stars

In [ ]:
import scipy.stats as sst

regr = sst.linregress(chart_stars['M']['mag'], chart_stars['V']['mag'])
regr

In [ ]:
import matplotlib.pyplot as plt
plt.plot(chart_stars['M']['mag'], chart_stars['V']['mag'], 'o')

plt.plot(chart_stars['M']['mag'], regr.intercept + regr.slope*chart_stars['M']['mag'].value, 'r')

plt.legend()

plt.show()

In [ ]:
import numpy as np

transformed = QTable (dict(
    id = stars['auid'],
    radec2000 = stars['radec2000'],
    V = regr.intercept *u.mag + regr.slope * stars['M']['mag'],
    Verr = np.sqrt((regr.intercept_stderr *u.mag)**2 + (stars['M']['mag']*regr.stderr)**2)
))
transformed = transformed[np.logical_and(np.logical_not(np.isnan(transformed['V'])), transformed['V'] < 16*u.mag)]
transformed.sort(['V'])
transformed

In [ ]:
center = SkyCoord(ra = reduced.wcs.wcs.crval[0] *u.deg, dec= reduced.wcs.wcs.crval[1] *u.deg)

from astroquery.simbad import Simbad
simbad = Simbad()

simbad.ROW_LIMIT = 100

# result = simbad.query_region(center, radius="0.7d")
# result

#center.to_string('hmsdms')
#reduced.header
pc = reduced.wcs.pixel_to_world(reduced.header['NAXIS1']/2, reduced.header['NAXIS1']/2)
pc.to_string('hmsdms'), center.to_string('hmsdms')


In [ ]:
from astroquery.vizier import Vizier

vizier = Vizier()
vizier.ROW_LIMIT = -1
result = vizier.query_region(center,
                             width=55*u.arcmin,
                             height=55*u.arcmin,
                             #catalog='II/336/apass9',
                             catalog='I/305',               # GSC 2.3
                             column_filters={'Vmag': '<16.5'})
cat_result = result[0]
cat_result['radec2000'] = SkyCoord(ra=cat_result['RAJ2000'],
                                   dec=cat_result['DEJ2000'])
cat_result.remove_columns(['RAJ2000', 'DEJ2000'])
cat_result

cat_result

In [ ]:

cat_stars = cat_result[cat_result['Class'] == 0]
cat_gals = cat_result[cat_result['Class'] == 1]
image.header['DATE-OBS']

In [ ]:
from astropy.visualization import AsinhStretch, ImageNormalize, MinMaxInterval
from matplotlib.patches import Circle


alpha = 0.001
interval = MinMaxInterval()
fig = plt.figure(figsize=(10.24, 10.24))
ax = plt.subplot(projection=reduced.wcs)
vmin, vmax = interval.get_limits(reduced.data)
norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=AsinhStretch(alpha))
ax.imshow(reduced.data, origin='lower', norm=norm)
for t in transformed:
    r = 2*(20 - t['V'].value)
    c = reduced.wcs.world_to_pixel(t['radec2000'])
    ax.add_patch(Circle(c, r, color='red', alpha=.5))
for s in cat_stars:
    r = 2*(20 - s['Vmag'])
    c = reduced.wcs.world_to_pixel(s['radec2000'])
    ax.add_patch(Circle(c, r, color='blue', alpha=.5))
for s in cat_gals:
    r = 2*(20 - s['Vmag'])
    c = reduced.wcs.world_to_pixel(s['radec2000'])
    ax.add_patch(Circle(c, r, color='yellow', alpha=.5))
plt.show()


In [ ]:
confirmed = join(transformed, cat_stars, keys='radec2000', join_funcs={'radec2000': join_skycoord(2 * u.arcsec)})
confirmed

In [ ]:
rr = sst.linregress(confirmed['V'], confirmed['Vmag'])

def pred(x):
    return rr.intercept*u.mag + rr.slope*x

def pred_err(x):
    return np.sqrt((rr.intercept_stderr*u.mag)**2 + (x*rr.stderr)**2)

predicted = pred(confirmed['V']) #rr.intercept*u.mag + rr.slope*confirmed['V']
err = pred_err(confirmed['V'])
good = np.abs(predicted - confirmed['Vmag']) < 3*err
rr


In [ ]:

plt.plot(confirmed['V'][good].value, confirmed['Vmag'][good].value, '.')
plt.plot(confirmed['V'][~good], confirmed['Vmag'][~good].value, 'o')
plt.plot(confirmed['V'][good], predicted[good], 'r')
plt.plot(confirmed['V'][good], (predicted+3*err)[good], 'g--')
plt.plot(confirmed['V'][good], (predicted-3*err)[good], 'g--')

# plt.legend()
plt.title(fr'$ R^2 = {rr.rvalue**2:.3g}$')
plt.show()

In [ ]:
confirmed[~good]

In [ ]:
from astropy.io import fits
from astropy.time import Time

f = fits.open("/srv/public/img/20240825/RZ_Lyr/Light/V/RZ_Lyr_Light_g200_e20_o10_b1x1_V_001.fits")
#Time(f[0].header['DATE-OBS']).jd,
f[0].header['FILTER']